# Table of Contents

1. [Problem Statement](#pstatement)
2. [Data Loading](#dload)
3. [Data Augmentation](#daugment)
4. [Model Building](#mbuild)
5. [Model Training](#mtrain)
6. [Performance Visualization](#pvis)
7. [Model Testing](#mtest)
8. [Performance Measures](#pmeasures)
9. [Qualitative Results](#quality)

# 1. Problem Statement <a name='pstatement'></a>

To implement an object detection pipeline using CNNs to localize and classifiy objects from the satellite imegry data.

**Dataset:**
Satellite Imagery Multi-vehicles Dataset (SIMD). It comprises 5,000 images of resolution 1024x768 and collectively contains 45,303 objects in 15 different classes of vehicles including cars, trucks, buses, long vehicles, various types of aircrafts and boats. The source images are taken from public satellite imagery available in Google Earth and contain images of multiple locations from seven countries.

Dataset can be downloaded from [here](https://drive.google.com/drive/folders/1vBKMtZIyy681oZz81TiWn05g1iAC1DsF?usp=sharing). You can access it only using your SEECS email address.

In [0]:
#Imports the print function from newer versions of python
from __future__ import print_function

%tensorflow_version 1.x
%matplotlib inline

TensorFlow 1.x selected.


In [0]:
import numpy as np

# 2. Data Loading<a name='dload'></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip "/content/drive/My Drive/Object_Detection_from_Satellite_Imagery_Data/SIMD.zip"

In [0]:
!unzip "/content/SIMS dataset/images.zip"

**Data Annotations:** With each image, the annotation is available as text file. The annotation format can be described as $(c, xi, yi, w, h)$, where $c$ is the object class starting from $0$, $(xi, yi)$ is the center of object and $w$, $h$ are width and height respectively. All these values are percentages to the actual image.

In [0]:
from glob import glob
import numpy as np
import cv2

def load_image(image_path): #Load an image from a file path
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def load_training_data():
    training_text_path = "/content/SIMS dataset/training.txt"
    training_text = np.genfromtxt(training_text_path, dtype=str, delimiter='\n')

    X_train, Y_train = [], []
    for image_name in training_text:
        image = load_image('/content/images/'+image_name[8:]) # image_name is like './images000...'
        if image.shape == (768, 1024, 3):
            label = np.genfromtxt('/content/images/'+image_name[8:].split('.')[0]+'.txt', delimiter=' ')
            X_train.append(image)
            Y_train.append(label)
        else: continue
    X_train = np.array(X_train, dtype="float") / 255.0
    return X_train, Y_train

def load_validation_data():
    validation_text_path = "/content/SIMS dataset/validation.txt"
    validation_text = np.genfromtxt(validation_text_path, dtype=str, delimiter='\n')

    X_valid, Y_valid = [], []
    for image_name in validation_text:
        image = load_image('/content/images/'+image_name[8:]) # image_name is like './images000...'
        label = np.genfromtxt('/content/images/'+image_name[8:].split('.')[0]+'.txt', delimiter=' ')
        X_valid.append(image)
        Y_valid.append(label)
    X_valid = np.array(X_valid, dtype="float") / 255.0
    return X_valid, Y_valid

def load_test_data():
    test_text_path = "/content/SIMS dataset/test.txt"
    test_text = np.genfromtxt(test_text_path, dtype=str, delimiter='\n')

    X_test, Y_test = [], []
    for image_name in test_text:
        image = load_image('/content/images/'+image_name[8:]) # image_name is like './images000...'
        label = np.genfromtxt('/content/images/'+image_name[8:].split('.')[0]+'.txt', delimiter=' ')
        X_test.append(image)
        Y_test.append(label)
    X_test = np.array(X_test, dtype="float") / 255.0
    return X_test, Y_test

def load_data():
    try: del X_train, Y_train, X_valid, Y_valid, X_test, Y_test
    except: pass

    print ("loading the training data..")
    X_train, Y_train = load_training_data()
    print ("done!")

    print ("loading the validation data..")
    X_valid, Y_valid = load_validation_data()
    print ("done!")

    print ("load the test data..")
    X_test, Y_test = load_test_data()
    print ("done!")

    return X_train, X_valid, X_test, Y_train, Y_valid, Y_test

In [0]:
(X_train, X_valid, X_test, Y_train, Y_valid, Y_test) = load_data()
print (len(X_train), len(X_valid), len(X_test))

loading the training data..
done!
loading the validation data..
done!
load the test data..


In [0]:
(X_train, X_valid, X_test, Y_train, Y_valid, Y_test) = load_data()

print ("X_train: ", X_train.shape, "\nY_train: ", Y_train.shape)
print ("X_valid: ", X_valid.shape, "\nY_valid: ", Y_valid.shape)
print ("X_test: ", X_test.shape, "\nY_test: ", Y_test.shape)